### 필요한 라이브러리 호출

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

### 문서 로드/분할 및 벡터 임베딩

In [2]:
# Chroma DB에서 문서 삭제
Chroma().delete_collection()

In [3]:
# PDF 파일 로드
loader = PyPDFLoader("data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load()

# 문서 분할
embeddings = HuggingFaceEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")
text_splitter = SemanticChunker(embeddings)
docs = text_splitter.split_documents(pages)

# LLM 선언
llm = ChatOllama(
    model = "basic_kanana",
    temperature = 0.7
)

# ChromaDB 선언
vectorstore = Chroma.from_documents(docs, embeddings)

# Retriever 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever = vectorstore.as_retriever(
        search_type = "mmr",
        search_kwargs = {"lambda_mult": 0.7, "fetch_k": 10, "k": 3}
    ),
    llm = llm
)

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 프롬프트 선언

In [4]:
# Langchain Hub에서 RAG 프롬프트 호출
prompt = hub.pull("rlm/rag-prompt")

# Retriever의 출력 결과물은 여러 개이므로 토큰 소모를 줄이기 위해 하나의 string으로 결합
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
# context와 question을 매개변수로 갖고 있는 것을 확인할 수 있음
# prompt.messages

### Chain 구축

In [5]:
rag_chain = (
    {"context": retriever_from_llm|format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 작동 결과 확인
# answer = rag_chain.invoke("국회의원의 의무는 뭐야?")
# print(answer)

국회의원의 의무는 다음과 같습니다. 첫째, 국회의원은 헌법이 정한 선거 절차에 따라 선출되어야 하며, 이를 통해 국민의 대표자로서 책임을 다해야 합니다. 둘째, 국회의원은 법률안을 제안하고 심의하며, 이를 통해 국가와 국민의 이익을 위해 일합니다. 셋째, 국회의원은 직무상 행한 발언과 표결에 대해 국회 외에서 책임을 지지 않지만, 그로 인해 발생하는 문제에 대해 책임을 집니다. 넷째, 국회의원은 청렴의 의무가 있으며 국가 이익을 최우선으로 고려해야 합니다.


In [ ]:
# Chain 구조 확인
# rag_chain.get_graph().print_ascii()

            +---------------------------------+          
            | Parallel<context,question>Input |          
            +---------------------------------+          
                    ***               ***                
                 ***                     ***             
               **                           ***          
+---------------------+                        **        
| MultiQueryRetriever |                         *        
+---------------------+                         *        
            *                                   *        
            *                                   *        
            *                                   *        
    +-------------+                     +-------------+  
    | format_docs |                     | Passthrough |  
    +-------------+*                    +-------------+  
                    ***               ***                
                       ***         ***                   
              